In [28]:
import network
import mnist_loader
import random
import numpy as np

In [8]:
training_data, validation_data, test_data = mnist_loader.load_data()

In [9]:
type(training_data)

tuple

In [11]:
training_data[1]

array([5, 0, 4, ..., 8, 4, 8])

In [16]:
print len(training_data[0])
print len(training_data[1])
print len(training_data)

50000
50000
2


In [14]:
from collections import defaultdict
T = defaultdict(list)

In [18]:
for img,digit in zip(training_data[0],training_data[1]):
    T[digit].append(img)

In [21]:
# making sure our d is what we expect it to be
print T.keys()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [22]:
# Checking the cardinality of T_d for each d
#for digit in T.keys():
for digit in range(0,10):
    print len(T[digit])

4932
5678
4968
5101
4859
4506
4951
5175
4842
4988


In [26]:
# fix d = 0 for now
# choose x1 and x2 uniformly at random
i1,i2 = random.sample(xrange(len(T[0])-1),2)
#print i1,i2
x1,x2 = T[0][i1],T[0][i2]
#print x1
#print x2

In [33]:
x = x2 - x1
#>>> x = numpy.array([0,1,2,3,0,5,6,0,8,9])
#>>> numpy.nonzero(x)
#(array([1, 2, 3, 5, 6, 8, 9]),)
S = set(np.nonzero(x)[0])

In [36]:
def jx(j,x):
    vec = np.zeros_like(x)
    vec[j] = x[j] 
    return vec

def select_random_j(S):
    return random.choice(tuple(S))

In [37]:
y = [0]*len(S)
y[0] = x[1]

num_S = len(S)
for i in range(1,num_S):
    j = select_random_j(S)
    S.remove(j)
    y[i] = y[i-1] + jx(j,x)